In [1]:
import yaml
import re
import numpy as np
import pandas as pd
from plasmistry import constants as const
from plasmistry.molecule import get_vib_energy

In [2]:
np.set_printoptions(precision=1, linewidth=130, edgeitems=5)

### Add constructors

In [3]:
def eval_constructor(loader, node):
    _str = loader.construct_scalar(node)
    return eval(_str)

yaml.add_constructor(u"!eval", eval_constructor)

In [4]:
def LT_constructor(loader, node):
    _list = loader.construct_sequence(node)
    A, B, C = _list
    return f"({A})*exp(({B})*Tgas**(-1/3)+({C})*Tgas**(-2/3))"

yaml.add_constructor(u"!LT", LT_constructor)

In [5]:
def Arr_constructor(loader, node):
    _list = loader.construct_sequence(node)
    A, b, E = _list
    return f"({A})*Tgas**({b})*exp(-({E})/Tgas)"

yaml.add_constructor(u"!Arr", Arr_constructor)

In [6]:
def H2_vib_energy_in_eV(*, v):
    return get_vib_energy('H2', quantum_number=v, minimum_is_zero=True)

def H2_vib_energy_in_eV(*, v):
    return get_vib_energy('H2', quantum_number=v, minimum_is_zero=True)

In [7]:
with open("test_0.yaml") as f:
    all_rctns = yaml.load(f)

In [8]:
all_rctns

[{'PATH': 'U:/__[FileSync]__/Coding/Python/CrossSectionFile/H2'},
 {'H2_vib': ['H2',
   'H2(v1)',
   'H2(v2)',
   'H2(v3)',
   'H2(v4)',
   'H2(v5)',
   'H2(v6)',
   'H2(v7)',
   'H2(v8)',
   'H2(v9)',
   'H2(v10)',
   'H2(v11)',
   'H2(v12)',
   'H2(v13)',
   'H2(v14)']},
 {'H2_vib': ['H2',
   'H2(v1)',
   'H2(v2)',
   'H2(v3)',
   'H2(v4)',
   'H2(v5)',
   'H2(v6)',
   'H2(v7)',
   'H2(v8)',
   'H2(v9)',
   'H2(v10)',
   'H2(v11)',
   'H2(v12)',
   'H2(v13)']},
 {'Molecule': {'mass': [44, 28]}},
 {'Rctn_block': {'type': 'excitation',
   'formula': 'E + A@ => E + B@',
   'threshold': [0.5155944840828292,
    1.001102962825446,
    1.4565254362278508,
    1.8818619042900435,
    2.2771123670120237,
    2.642276824393792,
    2.9773552764353477,
    3.2823477231366924,
    3.557254164497824,
    3.802074600518744,
    4.016809031199451,
    4.201457456539947,
    4.3560198765402305,
    4.480496291200301,
    0.4855084787426168,
    0.9409309521450216,
    1.3662674202072145,
    1.7615

In [9]:
class Reaction_block(object):

    def __init__(self, *, rctn_dict=None):
        super().__init__()
        self._formula = None
        self._kstr = None
        self._formula_list = None
        self._kstr_list = None
        self._type_list = None
        if rctn_dict is not None:
            self.rctn_dict = rctn_dict
            self._treat_rctn_dict()

    @property
    def _reactant_str_list(self):
        return [re.split(r"\s*<?=>\s*", _)[0] for _ in self._formula_list]

    @property
    def _product_str_list(self):
        return [re.split(r"\s*<?=>\s*", _)[1] for _ in self._formula_list]

    @property
    def size(self):
        assert len(self._formula_list) == len(self._kstr_list)
        return len(self._formula_list)

    def __add__(self, other):
        result = Reaction_block()
        result._formula_list = self._formula_list + other._formula_list
        result._kstr_list = self._kstr_list + other._kstr_list
        result._type_list = self._type_list + other._type_list
        return result

    def _treat_rctn_dict(self):
        self._formula = self.rctn_dict['formula']
        self._kstr = self.rctn_dict['kstr']

        assert ("zip" not in self.rctn_dict) or ("lambda" not in self.rctn_dict)
        # --------------------------------------------------------------------------------------- #
        if "where" in self.rctn_dict:
            assert isinstance(self.rctn_dict['where'], dict)
            if "abbr" in self.rctn_dict["where"]:
                reversed_abbr_dict = self.rctn_dict["where"]["abbr"][::-1]
                for _key_value in reversed_abbr_dict:
                    _key = list(_key_value.items())[0][0]
                    _value = str(list(_key_value.items())[0][1])
                    self._formula = self._formula.replace(_key, _value)
                    self._kstr = self._kstr.replace(_key, _value)
            if "vari" in self.rctn_dict["where"]:
                reversed_vari_dict = self.rctn_dict["where"]["vari"][::-1]
                for _key_value in reversed_vari_dict:
                    _key = list(_key_value.items())[0][0]
                    _value = f"({str(list(_key_value.items())[0][1])})"
                    self._formula = self._formula.replace(_key, _value)
                    self._kstr = self._kstr.replace(_key, _value)

        # --------------------------------------------------------------------------------------- #
        if "zip" in self.rctn_dict:
            _formula_list = []
            _kstr_list = []
            _zip_dict = self.rctn_dict['zip']
            _zip_number = len(_zip_dict[list(_zip_dict.keys())[0]])
            for _i in range(_zip_number):
                _formula = self._formula
                _kstr = self._kstr
                for _key in _zip_dict:
                    _formula = _formula.replace(_key, str(_zip_dict[_key][_i]))
                    _kstr = _kstr.replace(_key, str(_zip_dict[_key][_i]))
                _formula_list.append(_formula)
                _kstr_list.append(_kstr)
            self._formula_list = _formula_list
            self._kstr_list = _kstr_list
        if "lambda" in self.rctn_dict:
            lambda_func = self.rctn_dict['lambda']
            self._formula_list = lambda_func(self._formula)
            self._kstr_list = lambda_func(self._kstr)

        # --------------------------------------------------------------------------------------- #
        self._type_list = [self.rctn_dict["type"] for _ in range(self.size)]


In [10]:
class Cros_Reaction_block(Reaction_block):

    def __init__(self, *, rctn_dict=None):
        super().__init__(rctn_dict=rctn_dict)
        if rctn_dict is not None:
            self._threshold_list = self.rctn_dict["threshold"]

    def __add__(self, other):
        result = Cros_Reaction_block()
        result._formula_list = self._formula_list + other._formula_list
        result._kstr_list = self._kstr_list + other._kstr_list
        result._type_list = self._type_list + other._type_list
        result._threshold_list = self._threshold_list + other._threshold_list
        return result

    def generate_crostn_dataframe(self, *, factor=1):
        # _df = pd.DataFrame(index=range(self.size),
        #                    columns=["cs_key", "type", "threshold_eV", "cross_section"])
        _df = dict()
        _df["formula"] = self._formula_list
        _df["type"] = self._type_list
        _df["threshold_eV"] = self._threshold_list
        _df["cross_section"] = [np.vstack((np.loadtxt(_path)[:,0],
                                           np.loadtxt(_path)[:,1] * factor))
                                for _path in self._kstr_list]
        # # _df["cross_section"] = self._kstr_list
        # for _index, _path in zip(range(self.size), self._kstr_list):
        #     print(_index)
        #     print(_path)
        #     _df.loc[_index, "cross_section"] = np.loadtxt(_path)
        _df = pd.DataFrame(data=_df, index=range(self.size))
        return _df


In [11]:
rctn_considered = all_rctns[-1]['The reactions considered']

### Load reactions

In [12]:
rctn_block_0 = Cros_Reaction_block(rctn_dict=rctn_considered['electron reactions']['H2_ele_vib_rctn_forward'])
rctn_block_0.generate_crostn_dataframe()

,formula,type,threshold_eV,cross_section
0,E + H2 => E + H2(v1),excitation,0.515594,"[[0.0001, 0.00010016132706854269, 0.0001003229..."
1,E + H2 => E + H2(v2),excitation,1.001103,"[[0.4856084787426168, 0.48560864006968535, 0.4..."
2,E + H2 => E + H2(v3),excitation,1.456525,"[[0.9410309521450216, 0.9410311134720902, 0.94..."
3,E + H2 => E + H2(v4),excitation,1.881862,"[[1.3663674202072145, 1.366367581534283, 1.366..."
4,E + H2 => E + H2(v5),excitation,2.277112,"[[1.7616178829291946, 1.7616180442562632, 1.76..."
5,E + H2 => E + H2(v6),excitation,2.642277,"[[2.1267823403109634, 2.1267825016380315, 2.12..."
6,E + H2 => E + H2(v7),excitation,2.977355,"[[2.461860792352519, 2.461860953679587, 2.4618..."
7,E + H2 => E + H2(v8),excitation,3.282348,"[[2.7668532390538636, 2.7668534003809317, 2.76..."
8,E + H2 => E + H2(v9),excitation,3.557254,"[[3.0417596804149953, 3.0417598417420635, 3.04..."
9,E + H2 => E + H2(v10),excitation,3.802075,"[[3.286580116435915, 3.286580277762983, 3.2865..."


In [13]:
rctn_block_1 = Cros_Reaction_block(rctn_dict=rctn_considered['electron reactions']['H2_ele_vib_rctn_backward'])
rctn_block_1.generate_crostn_dataframe()

,formula,type,threshold_eV,cross_section
0,E + H2(v1) => E + H2,deexcitation,-0.515594,"[[0.00019353162758217746, 0.001025637313221694..."
1,E + H2(v2) => E + H2,deexcitation,-1.001103,"[[0.00019353162758228848, 0.001025637313221583..."
2,E + H2(v2) => E + H2(v1),deexcitation,-0.485508,"[[0.00019353162758217746, 0.001025637313221694..."
3,E + H2(v3) => E + H2,deexcitation,-1.456525,"[[0.00019353162758228848, 0.001025637313221583..."
4,E + H2(v3) => E + H2(v1),deexcitation,-0.940931,"[[0.00019353162758217746, 0.001025637313221694..."
5,E + H2(v3) => E + H2(v2),deexcitation,-0.455422,"[[0.00019353162758217746, 0.001025637313221694..."
6,E + H2(v4) => E + H2,deexcitation,-1.881862,"[[0.00019353162758228848, 0.001025637313221805..."
7,E + H2(v4) => E + H2(v1),deexcitation,-1.366267,"[[0.00019353162758228848, 0.001025637313221583..."
8,E + H2(v4) => E + H2(v2),deexcitation,-0.880759,"[[0.00019353162758217746, 0.001025637313221694..."
9,E + H2(v4) => E + H2(v3),deexcitation,-0.425336,"[[0.00019353162758217746, 0.001025637313221694..."


In [14]:
rctn_block_2 = Cros_Reaction_block(rctn_dict=rctn_considered['electron reactions']['H2_ele_dis_rctn_via_b'])
rctn_block_2.generate_crostn_dataframe(factor=1e-20)

,formula,type,threshold_eV,cross_section
0,E + H2 => E + H + H,EXCITATION,6.58,"[[6.58, 7.58, 8.58, 9.58, 10.6, 11.6, 12.6, 13..."
1,E + H2(v1) => E + H + H,EXCITATION,4.06,"[[4.06, 5.06, 6.06, 7.06, 8.06, 9.06, 10.1, 11..."
2,E + H2(v2) => E + H + H,EXCITATION,3.58,"[[3.58, 4.58, 5.58, 6.58, 7.58, 8.58, 9.58, 10..."
3,E + H2(v3) => E + H + H,EXCITATION,3.12,"[[3.12, 4.12, 5.12, 6.12, 7.12, 8.12, 9.12, 10..."
4,E + H2(v4) => E + H + H,EXCITATION,2.70,"[[2.7, 3.7, 4.7, 5.7, 6.7, 7.7, 8.7, 9.7, 10.7..."
5,E + H2(v5) => E + H + H,EXCITATION,2.30,"[[2.3, 3.3, 4.3, 5.3, 6.3, 7.3, 8.3, 9.3, 10.3..."
6,E + H2(v6) => E + H + H,EXCITATION,1.94,"[[1.94, 2.94, 3.94, 4.94, 5.94, 6.94, 7.94, 8...."
7,E + H2(v7) => E + H + H,EXCITATION,1.60,"[[1.6, 2.6, 3.6, 4.6, 5.6, 6.6, 7.6, 8.6, 9.6,..."
8,E + H2(v8) => E + H + H,EXCITATION,1.30,"[[1.3, 2.3, 3.3, 4.3, 5.3, 6.3, 7.3, 8.3, 9.3,..."
9,E + H2(v9) => E + H + H,EXCITATION,1.02,"[[1.02, 2.02, 3.02, 4.02, 5.02, 6.02, 7.02, 8...."


### Save dataframe to pickle

In [15]:
# rctn_block_0.generate_crostn_dataframe().to_pickle('rctn_dataframe_forward.pkl')

In [16]:
# rctn_block_1.generate_crostn_dataframe().to_pickle('rctn_dataframe_backward.pkl')

In [18]:
rctn_block_2.generate_crostn_dataframe(factor=1e-20).to_pickle("rctn_dataframe_H2_dis_via_b.pkl")